In [1]:
## Autoencoder : http://deeplearning.net/tutorial/dA.html ##
import numpy as np
import theano
import theano.tensor as T

from logistic import load_data

from theano.tensor.shared_randomstreams import RandomStreams
from utils import tile_raster_images


Using gpu device 0: GeForce GTX 960


In [2]:
class DenoisingAutoencoder(object):
    def __init__(self,input,rng,num_v,num_h,theano_rng=None):
        self.num_h = num_h
        self.num_v = num_v
        # setup random stream
        if not theano_rng:
            self.theano_rng = RandomStreams(rng.randint(2 ** 30))
        # init weight
        wval = np.asarray(rng.uniform(low=-4 * np.sqrt(6. / (num_h + num_v)),
                                      high=4 * np.sqrt(6. / (num_h + num_v)),
                                      size=(num_v,num_h)),dtype=theano.config.floatX)
        self.w = theano.shared(value=wval,name='w',borrow = True)
        # init visible layer bias
        self.bv = theano.shared(value = np.zeros(num_v,dtype=theano.config.floatX),name='bv',borrow=True)
        # init hidden layer bias
        self.bh = theano.shared(value = np.zeros(num_h,dtype=theano.config.floatX),name='bh',borrow=True)
        # setup weight hidden-output layer connections
        #  -> tied weights
        self.w_ = self.w.T
        self.x = input
        self.params = [self.w,self.bv,self.bh]
        
    def encode(self,x_):
        return T.nnet.sigmoid(T.dot(x_,self.w) + self.bh) ### Notice the use of bh here ###
    
    def decode(self,code):
        return T.nnet.sigmoid(T.dot(code,self.w_) + self.bv) ### Notice the use of bv here ###
    
    def loss(self,y,z):
        return - T.sum(self.x * T.log(z) + (1 - self.x) * T.log(1 - z), axis=1)

    def cost(self,y,z):
        return T.mean(self.loss(y,z))
    
    def corrupt_x(self,corruption_level):
        return self.theano_rng.binomial(size=self.x.shape, n=1,
                                        p=1 - corruption_level,
                                        dtype=theano.config.floatX) * self.x
   
    def step(self,corruption_level=0.,learning_rate=0.1):
        x_ = self.corrupt_x(corruption_level=corruption_level)
        y  = self.encode(x_)
        z  = self.decode(y)
        # setup cost, gradients and updates
        cost = self.cost(y,z)

        # gradients
        gparams = T.grad(cost,self.params)
        
        # updates
        updates = [ (param, param - (learning_rate*gparam) )
              for param,gparam in zip(self.params,gparams)]
        
        return (cost,updates)
            

In [5]:
x = T.matrix('x')
# numpy randomstate
rng = np.random.RandomState(12363)
da = DenoisingAutoencoder(rng=rng,input=x,num_v=28*28,num_h=500)

In [3]:
# Load MNIST data
datasets = load_data('mnist.pkl.gz')

batch_size = 20

train_set_x, train_set_y = datasets[0]
valid_set_x, valid_set_y = datasets[1]
test_set_x, test_set_y = datasets[2]

# compute number of minibatches for training, validation and testing
n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size

... loading data


In [6]:
index = T.lscalar('index')

cost,updates = da.step(corruption_level=0.3)
# compile train function
train = theano.function(inputs=[index],
                        outputs=cost,
                        updates=updates,
                       givens={ x : train_set_x[index*batch_size : (index+1)*batch_size]}
                       )

In [7]:
# actual training
for j in xrange(100):
    cost_iter = 0
    for i in xrange(n_train_batches):
        cost_iter +=  train(i)
    print 'cost per iteration : ',cost_iter/n_train_batches

cost per iteration :  81.8487316727
cost per iteration :  73.6065781754
cost per iteration :  70.9644064636
cost per iteration :  69.3590693268
cost per iteration :  68.3619637985
cost per iteration :  67.6654694885
cost per iteration :  67.183845697
cost per iteration :  66.8012939285
cost per iteration :  66.5456183334
cost per iteration :  66.3338180008
cost per iteration :  66.1104424957
cost per iteration :  65.9307809448
cost per iteration :  65.7940334183
cost per iteration :  65.6792003967
cost per iteration :  65.518634491
cost per iteration :  65.4138327393
cost per iteration :  65.3170308823
cost per iteration :  65.2860350082
cost per iteration :  65.1964902969
cost per iteration :  65.1203567871
cost per iteration :  65.0608457504
cost per iteration :  65.0062855972
cost per iteration :  64.9836480576
cost per iteration :  64.9050485611
cost per iteration :  64.853920163
cost per iteration :  64.8225509125
cost per iteration :  64.7627804687
cost per iteration :  64.735325

In [8]:
import PIL.Image as Image

# visualization
image = Image.fromarray(
        tile_raster_images(X=da.w.get_value(borrow=True).T,
                           img_shape=(28, 28), tile_shape=(10, 10),
                           tile_spacing=(1, 1)))
image.save('filters_corruption_30.png')
